<a href="https://colab.research.google.com/github/Arya1790/RAG/blob/main/RAG_langchain_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Langchain: Langchain expression language

In [1]:
!pip install langchain
!pip install langchain_community
!pip install pypdf
!pip install faiss-gpu
!pip install --upgrade --quiet  google-generativeai langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.9 MB/s eta 0:00:00


In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import os
from google.colab import userdata
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

In [3]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [4]:
loader = PyPDFLoader("/content/drive/MyDrive/genAI/retrieval-augmented-generation-for-knowledge-intensive-nlp-tasks.pdf")

In [5]:
pages = loader.load_and_split()
len(pages)

23

In [6]:
gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

In [8]:
faiss_index = FAISS.from_documents(pages, gemini_embeddings)

docs = faiss_index.similarity_search("what is RAG?", k=2)

len(docs)

2

In [9]:
retriever = faiss_index.as_retriever(k=3)

In [10]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

In [11]:
prompt = ChatPromptTemplate.from_template(template)
parser = StrOutputParser()

In [12]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [14]:
chain.invoke("what is Retrieval-Augmented Generation?")

'Retrieval-Augmented Generation (RAG) is a general-purpose fine-tuning approach for models that combine pre-trained parametric and non-parametric memory for language generation. \n\nIt consists of:\n\n* **Parametric memory:** A pre-trained seq2seq model (e.g., BART)\n* **Non-parametric memory:** A dense vector index of a knowledge source (e.g., Wikipedia), accessed with a pre-trained neural retriever (e.g., Dense Passage Retriever - DPR)\n\nRAG models are trained end-to-end, jointly learning the retriever and generator. The retriever finds relevant documents based on the input, and the generator uses both the input and retrieved documents to produce the output. \n'